In [9]:
import numpy as np
import pandas as pd
import random

random.seed(42)
np.random.seed(42)

In [226]:
def make_ts(output_len, state_change, percent_talking=1.0, noise_level=0):
    assert state_change.ndim == 2, "state_change have more than 2 dims"
    assert state_change.shape[0] == state_change.shape[1], "state_change is not square"
    assert all([sum(x) == 1 for x in state_change]), "All rows do not sum to 1"
    assert percent_talking >= 0.0 and percent_talking <= 1.0, "percent_talking is not a probability"
    
    state = 0
    choices = state_change.shape[0]
    ts = np.random.rand(choices, output_len) * noise_level
    all_clean = np.random.rand(output_len) * 0.001
    
    for i in range(output_len):
        state = np.random.choice(choices, p=state_change[state])
        if random.random() <= percent_talking:
            ts[state][i] = 1
            all_clean[i] = 1
    return ts, all_clean, ts.sum(axis=0)

splits, all_clean, all_raw = make_ts(100_000, np.array([[0.5, 0.5],[0.5,0.5]]), percent_talking= .99, noise_level=10)

In [227]:
import statsmodels.api as sm


# https://www.datainsightonline.com/post/cross-correlation-with-two-time-series-in-python
# from scipy import signal
def ccf_values(series1, series2):
    p = series1
    q = series2
    p = (p - np.mean(p)) / (np.std(p) * len(p))
    q = (q - np.mean(q)) / (np.std(q))
    c = np.correlate(p, q, 'full')
    return c


def ccf_calc(sig1, sig2):
    corr = sm.tsa.stattools.ccf(sig2, sig1, adjusted=False, fft=True)

    # Remove padding and reverse the order
    #print(corr)
    return corr[0:(len(sig2)+1)]#[::-1]


def cross_cor(ts1, ts2, debug=False, max_offset=300, only_positive=True):
    ccf = ccf_calc(ts1, ts2)
    #print(ccf)
    best_cor = max(ccf)
    best_lag = np.argmax(ccf)

    if debug:
        print('best cross correlation: ' + str(best_cor) + " at time lag: " + str(best_lag))
        print(len(ccf))
        print(ccf)
        ccf_plot(range(len(ccf)), ccf)
    return best_cor, best_lag

In [228]:
print(cross_cor(all_raw, all_clean))
print(cross_cor(splits[0], all_clean))
print(cross_cor(splits[1], all_clean))

(np.float64(0.013217086103072598), np.int64(3625))
(np.float64(0.01144250901790135), np.int64(4439))
(np.float64(0.011575890117305336), np.int64(17773))
